> A project to analyze hacker news stories using nbdbt

## Analysis

In [1]:
#| echo: false
%reload_ext nbdbt.dbt_cellmagic

In [2]:
#| echo: false
%dbtconfig -p ../hn_whos_hiring -n notebooks/analysis.ipynb

### Raw HN Source

> This is the raw data for all Hacker News posts

It is sourced from the Google's Big Query Public Datases
and accessible as 
```
bigquery-public-data.hacker_news.full
```

It seems to be updated on a regular basis *(see timestamp of latest post)*.

In [3]:
%%dbt -a raw_sources analyses/raw_hn_source.sql
select *
from {{ source('public_datasets', 'full_stories') }}
order by timestamp desc


In [4]:
df = raw_sources.ref(10); df.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,POSIX sh parameter expansion cheat sheet:<p><a...,None,jwilk,NaN,1656580359,2022-06-30 09:12:39+00:00,comment,31930208,31928736.0,NaN,NaN,None
1,None,None,That also ate a lot of small healthy business ...,None,Existenceblinks,NaN,1656580348,2022-06-30 09:12:28+00:00,comment,31930207,31929941.0,NaN,NaN,None
2,None,None,Does anyone actually like using JIRA? Or Confl...,None,gaff33,NaN,1656580322,2022-06-30 09:12:02+00:00,comment,31930206,31929941.0,NaN,NaN,None
3,None,None,Not at all is the issue. IIRC svn checkout is ...,None,masklinn,NaN,1656580321,2022-06-30 09:12:01+00:00,comment,31930205,31929148.0,NaN,NaN,None
4,None,None,"That&#x27;s because, if the chip uses 20% less...",None,tintedfireglass,NaN,1656580316,2022-06-30 09:11:56+00:00,comment,31930204,31925613.0,NaN,NaN,None


The column names have some descriptions, but might need to be standardized.

In [5]:
df.columns.values

array(['title', 'url', 'text', 'dead', 'by', 'score', 'time', 'timestamp',
       'type', 'id', 'parent', 'descendants', 'ranking', 'deleted'],
      dtype=object)

### Exploratory Data Analysis 
> based on a sample size of 10 latest posts

* Check if time and timestamp contain the same info and one column can be eliminated

In [6]:
from datetime import datetime
import pandas as pd

In [7]:
df['newtimestamp'] = pd.to_datetime(df['time'].astype(float), unit='s',origin='unix', utc=True)

In [8]:
df[df['newtimestamp'] == df['timestamp']].any(axis=None)

True

Looks like they are one and the same.

### EDA Questions

* Check total record count

In [9]:
%%dbt -a tot_rec analyses/count_hn_source.sql
select count(*) as rec_count 
from {{ source('public_datasets', 'full_stories') }}


In [10]:
tot_rec_df = tot_rec.ref()

In [11]:
total_records = tot_rec_df.iloc[0].rec_count

In [12]:
total_records

31930207

* Check if any id, by, time, timestamp, type are null 

In [13]:
%%dbt -a null_field_counts analyses/null_field_counts.sql
with hn_posts
as (
select
     `by` as author,
     * except(`by`)
from {{ source('public_datasets', 'full_stories') }}
)
select 
   'id' as field, 
    count(*) as null_count,
from hn_posts
where id is null
union all
select 
   'author' as field, 
    count(*) as null_count,
from hn_posts
where author is null
union all
select 
   'time' as field, 
    count(*) as null_count,
from hn_posts
where time is null
union all
select 
   'timestamp' as field, 
    count(*) as null_count,
from hn_posts
where timestamp is null
union all
select 
   'type' as field, 
    count(*) as null_count,
from hn_posts
where type is null
union all
select 
   'dead' as field, 
    count(*) as null_count,
from hn_posts
where dead is null



In [14]:
null_df = null_field_counts.ref()

In [15]:
null_df['pct'] = null_df['null_count']/total_records

In [16]:
null_df

,field,null_count,pct
0,type,0,0.000000
1,id,0,0.000000
2,author,947682,0.029680
3,time,26818,0.000840
4,timestamp,26818,0.000840
5,dead,30560332,0.957098


| So `type` and `ids` all have values, 
| but there are entries with no `time` or `timestamp` (very small, less than 0.1 percent)
| and there are entries with no `author` (around 3 percent)
| and 96 percent have null values for the `dead` field.

* Check for unique values of `dead`

In [17]:
%%dbt -a dead_type_counts analyses/dead_types_counts.sql
select dead as dead_type, count(*) as dead_count
from {{ source('public_datasets', 'full_stories') }}
group by dead 
order by dead_count desc

In [18]:
dead_types_df = dead_type_counts.ref()
dead_types_df['pct'] = dead_types_df['dead_count']/total_records

In [19]:
dead_types_df

,dead_type,dead_count,pct
0,None,30560332,0.957098
1,True,1369875,0.042902


* Sample dead values

In [20]:
%%dbt -a dead_rows analyses/dead_rows.sql
select * 
from {{ source('public_datasets','full_stories') }}
where dead is not null
limit 10

In [21]:
dead_rows_df = dead_rows.ref()

In [22]:
dead_rows_df.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,"Thank you, I'm not afraid to say I've done som...",True,TheTruthIsHere,NaN,1345996009,2012-08-26 15:46:49+00:00,comment,4435323,4434940,NaN,NaN,None
1,None,None,"&gt;&gt;Lennart Poettering, though, described ...",True,nix23,NaN,1614359471,2021-02-26 17:11:11+00:00,comment,26277078,26275289,NaN,NaN,None
2,None,None,A very awesome blog post. We are really gratef...,True,Kirschbaum231,NaN,1623305806,2021-06-10 06:16:46+00:00,comment,27457407,27455706,NaN,NaN,None
3,None,None,Well played.,True,frozenport,NaN,1567159573,2019-08-30 10:06:13+00:00,comment,20837202,20835655,NaN,NaN,None
4,None,None,"I guess if you&#x27;re African American, this ...",True,PythonDeveloper,NaN,1391736249,2014-02-07 01:24:09+00:00,comment,7194243,7194131,NaN,NaN,None


In [23]:
%%dbt -a not_dead_rows analyses/not_dead_rows.sql
select * 
from {{ source('public_datasets','full_stories') }}
where dead is null
limit 10

In [24]:
not_dead_rows_df = not_dead_rows.ref()

In [25]:
not_dead_rows_df.head()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,Or companies which can afford research fellows...,None,mc32,NaN,1582748998,2020-02-26 20:29:58+00:00,comment,22426980,22426627,NaN,NaN,None
1,None,None,I see lots of advice about stocking up on basi...,None,bronco21016,NaN,1582749003,2020-02-26 20:30:03+00:00,comment,22426982,22425593,NaN,NaN,None
2,None,None,It is way safer and quicker to do the return t...,None,jfkebwjsbx,NaN,1582749009,2020-02-26 20:30:09+00:00,comment,22426984,22426541,NaN,NaN,None
3,None,None,"I learned a new word today <a href=""https:&#x2...",None,ipnon,NaN,1582749038,2020-02-26 20:30:38+00:00,comment,22426987,22426514,NaN,NaN,None
4,None,None,"&gt; To be honest, I have never seen sports as...",None,obmelvin,NaN,1582749043,2020-02-26 20:30:43+00:00,comment,22426989,22424528,NaN,NaN,None


* Check for unique values of `type`  

In [26]:
%%dbt -a type_counts analyses/types_counts.sql
select type as type, count(*) as type_count
from {{ source('public_datasets', 'full_stories') }}
group by type 
order by type_count desc

In [27]:
types_df = type_counts.ref()
types_df['pct'] = dead_types_df['dead_count']/total_records

In [28]:
types_df

,type,type_count,pct
0,comment,27351834,0.957098
1,story,4547262,0.042902
2,job,15502,NaN
3,pollopt,13633,NaN
4,poll,1976,NaN


## Standardization
> Standardize column names and types so downstream transformations don't have to deal with that

```
WITH stories AS (
    SELECT
        * EXCEPT (`by`),
        `by` AS authored_by,
        EXTRACT(YEAR FROM time_ts) AS year_published
    FROM {{ source('hacker_news','stories') }}

)

```

In [29]:
%%dbt -a hn_posts models/hn_posts.sql
with stories as (
  select
    * except (`by`),
    `by` as submitter_id,
  from {{ source('public_datasets', 'full_stories') }}
),
latest_stories as (
  select 
     id as post_id, 
     title,
     url,
     submitter_id,
     text as content,
     timestamp as submit_timestamp,
     dead as post_status,  
     score as post_score,
     cast(parent as numeric) as parent_id,
     type as post_type,
   from stories
   order by submit_timestamp desc
)
select *
from latest_stories


In [30]:
project_dir = '../hn_whos_hiring'
profiles_dir = '~/.dbt'

In [34]:
model = faldbt.list_models()[0]

In [33]:
source = faldbt.sources[0]

In [38]:
src_node = source.node
mdl_node = model.node

In [35]:
# faldbt._profile_target

In [82]:
from fal import FalDbt
import faldbt.lib as fallib

def get_schema(source_or_model, project_dir, profiles_dir):
    node = source.node
    faldbt = FalDbt(project_dir,profiles_dir)

    adapter = fallib._get_adapter(faldbt.project_dir,faldbt.profiles_dir,faldbt._profile_target)

    # adapter.type() == 'bigquery'
    if adapter.type() != 'bigquery':
        raise NotImplementError('No support yet for any other adapter except BigQuery')
        return None

    relation = fallib._get_target_relation(source_or_model.node, faldbt.project_dir,faldbt.profiles_dir,profile_target=faldbt._profile_target)

    info_schema = relation.information_schema()

    column_schema = info_schema.from_relation(relation,'COLUMNS')

    column_table = column_schema.render()

    table_name = relation.table

    fetch_schema_sql = f'''
    with schema_columns as
    ( select *
    from {column_table}
    where table_name = '{table_name}'
    )
    select * 
    from schema_columns
    '''

    _, result = fallib._execute_sql(project_dir, profiles_dir, fetch_schema_sql, faldbt._profile_target)
    return result

In [83]:
faldbt.list_sources()

[DbtSource(name='public_datasets', tests=[], status='skipped')]

In [84]:
faldbt.list_models()

[DbtModel(name='hn_posts', alias='hn_posts', unique_id='model.hn_whos_hiring.hn_posts', columns={}, tests=[], status='success')]

In [85]:
faldbt.list_sources()[0].unique_id

'source.hn_whos_hiring.public_datasets.full_stories'

In [86]:
get_schema(faldbt.list_sources()[0],faldbt.project_dir, faldbt.profiles_dir)

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name
0,bigquery-public-data,hacker_news,full,title,1,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
1,bigquery-public-data,hacker_news,full,url,2,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
2,bigquery-public-data,hacker_news,full,text,3,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
3,bigquery-public-data,hacker_news,full,dead,4,YES,BOOL,NEVER,None,None,NO,None,NO,NO,NaN,NULL
4,bigquery-public-data,hacker_news,full,by,5,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
5,bigquery-public-data,hacker_news,full,score,6,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL
6,bigquery-public-data,hacker_news,full,time,7,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL
7,bigquery-public-data,hacker_news,full,timestamp,8,YES,TIMESTAMP,NEVER,None,None,NO,None,NO,NO,NaN,NULL
8,bigquery-public-data,hacker_news,full,type,9,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
9,bigquery-public-data,hacker_news,full,id,10,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL


In [87]:
get_schema(faldbt.list_models()[0],faldbt.project_dir, faldbt.profiles_dir)

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name
0,hn-whos-hiring,00dev,hn_posts,post_id,1,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL
1,hn-whos-hiring,00dev,hn_posts,title,2,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
2,hn-whos-hiring,00dev,hn_posts,url,3,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
3,hn-whos-hiring,00dev,hn_posts,submitter_id,4,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
4,hn-whos-hiring,00dev,hn_posts,content,5,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL
5,hn-whos-hiring,00dev,hn_posts,submit_timestamp,6,YES,TIMESTAMP,NEVER,None,None,NO,None,NO,NO,NaN,NULL
6,hn-whos-hiring,00dev,hn_posts,post_status,7,YES,BOOL,NEVER,None,None,NO,None,NO,NO,NaN,NULL
7,hn-whos-hiring,00dev,hn_posts,post_score,8,YES,INT64,NEVER,None,None,NO,None,NO,NO,NaN,NULL
8,hn-whos-hiring,00dev,hn_posts,parent_id,9,YES,NUMERIC,NEVER,None,None,NO,None,NO,NO,NaN,NULL
9,hn-whos-hiring,00dev,hn_posts,post_type,10,YES,STRING,NEVER,None,None,NO,None,NO,NO,NaN,NULL


In [57]:
%%dbt -a hn_table_schema analyses/meta/hn_posts_schema.sql
select *
from bigquery-public-data.hacker_news.INFORMATION_SCHEMA.COLUMNS
where table_name = 'full'

In [ ]:
hn_table_schema_df = hn_table_schema.ref()

In [ ]:
hn_table_schema_df